In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import numpy as np
import numpy.random as rng
from sklearn import datasets, neighbors
import seaborn as sns
import pandas as pd
np.set_printoptions(precision=3) # limit precision when printing arrays

In [ ]:
# Sample Decision Tree Classifier
from sklearn import datasets
from sklearn import metrics
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

# To suppress boring warnings...
from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning

In [ ]:
### This is the code block to load the csv files for training/testing.

datasets  = {}

df = pd.read_csv('./ionosphere_new.csv')
X, y = df.drop('class', axis=1),  df['class']
datasets["ionosphere"] = (X,y)

df = pd.read_csv('./steelplates_new.csv')
X, y = df.drop('Class', axis=1),  df['Class']
datasets["steelplates"] = (X,y)

df = pd.read_csv('./banknotes_new.csv')
X, y = df.drop('Class', axis=1),  df['Class']
datasets["banknotes"] = (X,y)

# TODO: set the number of trials required to evaluate the performance of a classifier.
nTrials = 10

In [ ]:
import numpy as np
import random

@ignore_warnings(category=ConvergenceWarning)
def test(dataname, classifier, controlName, controlOptions, noXtics=False):
    random.seed(100)  # Python's built-in random module
    np.random.seed(100)
    
    X,y = datasets[dataname]

    
    org_ctl_options = controlOptions

    scores = np.zeros(shape=(nTrials, len(controlOptions)))
    for t in range(nTrials):
        # TODO: perform train test split with the 50:50 ratio.
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=t)

        # TODO: standardize the value range of each feature in the dataset.
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

        for i,option in enumerate(controlOptions):
            model = classifier(**{controlName: option})
            # TODO: perform model training.
            model.fit(X_train, y_train)
            
            scores[t,i] += model.score(X_test, y_test)

    # Draw the box plots to visually display the model performance.
    if controlName in ['alpha']: controlOptions = np.log10(controlOptions) # for semilogx
    elif controlName in ['kernel']: controlOptions = [1, 2, 3, 4]
    a = plt.boxplot(scores, positions=controlOptions, showfliers=False)
    if noXtics:
        plt.xticks([])
    else:
        if controlName in ['kernel']:
            plt.xticks(controlOptions, org_ctl_options)
        plt.xlabel(controlName)
    plt.title(dataname + " " + type(model).__name__[:12])
    print("{0:.2f} was the best {1} on {2}".format(np.max(scores.mean(axis=0)),type(model).__name__[:12],dataname))
    return(plt)

In [ ]:
def testSeveralDatas(classifier, controlName, options):
    plt.figure(figsize=(12, 4))
    plt.subplot(131)
    test("banknotes", classifier, controlName, options)
    plt.subplot(132)
    test("steelplates", classifier, controlName, options)
    plt.subplot(133)
    test("ionosphere", classifier, controlName, options)
    plt.savefig(type(classifier()).__name__ + ".png")

In [ ]:
testSeveralDatas(KNeighborsClassifier,"n_neighbors", range(1,6,1))

1.00 was the best KNeighborsCl on banknotes
0.98 was the best KNeighborsCl on steelplates
0.85 was the best KNeighborsCl on ionosphere


In [ ]:
testSeveralDatas(DecisionTreeClassifier,"max_depth", range(1,11,1))

0.98 was the best DecisionTree on banknotes
1.00 was the best DecisionTree on steelplates
0.89 was the best DecisionTree on ionosphere


In [ ]:
testSeveralDatas(RandomForestClassifier, "max_depth", range(1,11,1))

0.99 was the best RandomForest on banknotes
0.98 was the best RandomForest on steelplates
0.93 was the best RandomForest on ionosphere


In [ ]:
testSeveralDatas(SVC, "kernel" , ['linear','poly', 'rbf', 'sigmoid'])

1.00 was the best SVC on banknotes
1.00 was the best SVC on steelplates
0.94 was the best SVC on ionosphere


In [ ]:
### This code block can be used to generate the ROC curves of the random forest classifier on one dataset. We set the maximum depth to 5.

from sklearn.metrics import RocCurveDisplay
import numpy as np
import random

random.seed(100)
np.random.seed(100)

# commented and uncommented the different datasets to display the ROC curves
#X, y = datasets["ionosphere"]
#X, y = datasets["steelplates"]
X, y = datasets["banknotes"]

# TODO: perform train test split with the 50:50 ratio.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=100)

# TODO: standardize the value range of each feature in the dataset.
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Initialize and train classifier
rf_clf = RandomForestClassifier(max_depth=5)
svc_clf = SVC(kernel='rbf', probability=True)

# TODO: perform model training.
rf_clf.fit(X_train, y_train)
svc_clf.fit(X_train, y_train)

roc_disp = RocCurveDisplay.from_estimator(rf_clf, X_test, y_test)
roc_disp = RocCurveDisplay.from_estimator(svc_clf, X_test, y_test)
plt.show()